
Programmieren 3 - Wissenschaftliche Anwendungen

Peter Rösch, Fakultät für Informatik

Hochschule Augsburg, 2023 / 2024

# Einführung

## Wissenschaftliche Anwendungen - Überblick

Einen guten Überblick über stabile Pakete finden Sie auf der Seite https://scientific-python.org/specs/core-projects.

Ziel der heutigen Veranstaltung ist es, einen Überblick einen Teil der genannten Pakete zu erhalten.

## Buch zu wissenschaftlichen Anwendungen mit Python

Viele Beispiele zur Verwendung von IPython für wissenschaftliche Anwendungen finden Sie in folgendem Buch von Cyrille Rossant:  [IPython Interactive Computing and Visualization Cookbook](https://ipython-books.github.io/). Sie können das komplette Buch über die Bibliothek online lesen, hier ist der [Link](https://learning.oreilly.com/library/view/ipython-interactive-computing/9781785888632).

# Symbolische Berechnungen mit sympy

Idee: Symbolische Berechnungen statt Numerik. Großes Vorbild: [Mathematica](https://www.wolfram.com/mathematica).

Weitergehende Informationen zu sympy finden Sie auf der [Homepage](http://sympy.org/en/index.html).

Zunächst müssen die Symbole festgelegt werden, mit denen sympy rechnen soll:

In [ ]:
import sympy

sympy.init_printing(use_latex=True)
x, y, z = sympy.symbols("x y z")
k, m, n = sympy.symbols("k m n", integer=True)
f, g, h = map(sympy.Function, "fgh")

Mit diesen Symbolen können dann Ausdrücke formuliert werden:

In [ ]:
eq = (x - y) ** 3 * (x + 1) ** 2
eq

Der Ausdruck oben kann jetzt ausmultipliziert werden:

In [ ]:
sympy.expand(eq)

Sympy löst Integrale symbolisch:

In [ ]:
f = sympy.cos(x) ** 3
sympy.Integral(f, x)

In [ ]:
g = sympy.integrate(f, x)
g

Probe:

In [ ]:
h = sympy.diff(g, x)
h

Ist das wirklich die ursprüngliche Funktion *f* ?

In [ ]:
sympy.simplify(h)

Sympy kann auch Differentialgleichungen lösen:

In [ ]:
# f muss neu initialisiert werden
f = sympy.Function("f")
eqn = sympy.Eq(sympy.Derivative(f(x), x, x) + 9 * f(x), 1)
# Anzeige von Formeln mit display
from IPython.display import display

display(eqn)
# Berechnen und Anziegen der Loesung
sympy.dsolve(eqn, f(x))

## Übung (5 Minuten)

Gehen Sie auf die Seite https://sympygamma.com und geben Sie folgende Rechenaufgabe ein:

    integrate(sin(x)**3, x)
    
1. Was fällt Ihnen an der Ausgabe auf?
1. Kommt Ihnen [diese Rechnung](https://docs.sympy.org/latest/modules/physics/units/examples.html) bekannt vor?

Python kann mit beliebiger Genauigkeit rechnen:

## Rechnen mit beliebiger Genauigkeit

Das Paket [mpmath](http://mpmath.org) erlaubt es, numerische Berechnungen mit (fast) beliebiger Genauigkeit durchzuführen.

In [ ]:
import mpmath

mpmath.mp.dps = 60
mpmath.mp.pretty = True
# direkte Ausgabe
print("p0:", mpmath.pi)
# aus dem arcus sinus
p1 = 2 * mpmath.asin(1)
print("p1:", p1)
# Numerische Integration einer Gauss-Kurve
p2 = (
    mpmath.quad(
        lambda x: 100 * mpmath.exp(-(x**2)), [-mpmath.inf, mpmath.inf]
    )
    ** 2
)
print("p2:", p2)

# numpy und numba - Vertiefung

Numpy überlädt Operatoren so, dass sie elementweise funktionieren. Das zugrundeliegende Konzept ist das der "universal function" [ufunc](https://numpy.org/doc/stable/reference/ufuncs.html). 

In [ ]:
import numpy as np

# Skalare
a, b = 33, 44
print(f"a+b: {np.add(a, b)}")

In [ ]:
# Vektoren
a = np.random.uniform(-3, 3, 3)
b = np.random.uniform(-3, 0, 3)
print(f"a+b: {np.add(a, b)}")

In [ ]:
# Matrizen
a = np.random.uniform(-3, 3, (3, 3))
b = np.random.uniform(-3, 0, (3, 3))
print(f"a+b:\n{np.add(a, b)}")

In [ ]:
# ufunc - Broadcasting und Typ-Anpassung
a = np.abs(np.random.normal(size=(4, 3)))
b = np.array([1, 2, 3], dtype=np.uint8)
print(f"a:\n{a}\na+b({(a+b).dtype})\n{a+b}:")

In [ ]:
%timeit c = a + b

Frage: Wie können wir das mit *numba* nachbilden?

Wie üblich wollen wir möglichst wenig Aufwand investieren ...

Lösung: [vectorize / guvectorize](https://numba.pydata.org/numba-doc/latest/user/vectorize.html).

In [ ]:
import numba

In [ ]:
@numba.vectorize
def numba_add(x, y):
    return x + y

Für welche Datentypen wurde optimierter Code generiert?

In [ ]:
numba_add.types

In [ ]:
%timeit c = numba_add(a, b)

In [ ]:
numba_add.types

In [ ]:
d = numba_add(a, a)
numba_add.types

# Interpolation

Interpolationsmethoden erlauben es, Werte, zwischen bekannten Messwerten liegen, unter Verwendung eines bestimmten Modells zu 'raten'.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib ipympl

In [ ]:
N = 16
np.random.seed(2019)
s = np.cumsum(np.random.uniform(-1, 1, N))
x = range(0, N)

In [ ]:
from scipy import interpolate

f = interpolate.interp1d(x, s, kind="cubic")
xN = np.arange(0, N - 1, 0.01)
yN = f(xN)
ax = plt.figure().add_subplot(111)
ax.plot(x, s, "ro")
ax.plot(xN, yN)

Durch Variation des Parameters *kind* können unterschiedliche Interpolationsverfahren ausgewählt werden.

In [ ]:
help(interpolate.interp1d)

# Digitale Bildverarbeitung

Die Python Imaging Library (PIL) erlaubt es auf einfache Art und Weise, Bilder zu laden, zu manipulieren und anzuzeigen:

In [ ]:
import math
from PIL import Image, ImageFilter

In [ ]:
im = Image.open("cameraman.png")
ax = plt.figure().add_subplot(111)
ax.imshow(im, cmap=plt.cm.gray)

In [ ]:
out_im = im.rotate(-45).filter(ImageFilter.BLUR)
ax = plt.figure().add_subplot(111)
ax.imshow(out_im, cmap=plt.cm.gray)

Bilder können in numpy-Arrays umgewandelt werden:

In [ ]:
a = np.asarray(im, dtype=np.ubyte)
# Schwellwert
b = np.where(a < 100, 1, 0)
ax = plt.figure().add_subplot(111)
ax.imshow(b, cmap=plt.cm.gray)

Das Paket *scipy.ndimage* gibt dem Entwickler mehr Kontrolle darüber, was passiert und enthält fortgeschrittene Bildverarbeitungs-Methoden wie z.B. Morphologie:

In [ ]:
import scipy.ndimage

In [ ]:
c = scipy.ndimage.binary_opening(b)
ax = plt.figure().add_subplot(111)
ax.imshow(c, cmap=plt.cm.gray)

Die Anwendung einer Filter-Maske nennt man Faltung ('convolution'). Beispiel: Box-Filter:

In [ ]:
box_filter = np.ones(shape=(5, 5))
box_filter /= np.sum(box_filter)
print("box filter:\n", box_filter)
a_smoothed = scipy.ndimage.convolve(a, box_filter)
ax = plt.figure().add_subplot(111)
ax.imshow(a_smoothed, cmap=plt.cm.gray)

In [ ]:
from skimage import data, filters

im = data.camera()
edges = filters.sobel(im)
ax = plt.figure().add_subplot(111)
ax.imshow(edges, cmap=plt.cm.gray)

Das Paket *OpenCV* bietet hoch optimierte Funktionen aus dem Bereich Computer Vision und erlaubt auf einfache Art und Weise den Zugriff auf Kameras. Schließen Sie eine Kamera an, machen Sie diese ggf. für die VM verfügbar und starten Sie das folgende Beispiel.

In [ ]:
import cv2

cap = cv2.VideoCapture(0)
FPS = 25
while cv2.waitKey(1000 // FPS) & 0xFF != ord("q"):
    success, frame = cap.read()
    if success:
        cv2.imshow("Orignal", frame)
        grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(grey, 120, 255, cv2.THRESH_BINARY_INV)
        cv2.imshow("Result", thresh)
cap.release()
cv2.destroyAllWindows()

Frage: Wie könnte man diesen Code lesbarer machen?

In [ ]:
import cv2


class CameraFrames:
    """Context manager for Camera frame reading"""

    def _frame_generator(self):
        while cv2.waitKey(1000 // self._frames_per_second) & 0xFF != ord("q"):
            success, frame = self._capture.read()
            if success:
                yield frame.copy()

    def __enter__(self):
        self._capture = cv2.VideoCapture(0)
        self._frames_per_second = 25
        while not self._capture.read()[0]:
            pass

        gen = self._frame_generator()
        next(gen)
        return gen

    def __exit__(self, exc_type, exc_value, tb):
        self._capture.release()
        cv2.destroyAllWindows()
        if exc_type is not None:
            traceback.print_exception(exc_type, exc_value, tb)

In [ ]:
def process_camera_frames(frame_function, *args, **kwargs):
    with CameraFrames() as frames:
        for frame in frames:
            frame_function(frame, *args, **kwargs)

In [ ]:
def frame_threshold(frame, threshold=128, gval=255):
    cv2.imshow("Orignal", frame)
    grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(grey, threshold, gval, cv2.THRESH_BINARY_INV)
    cv2.imshow("Result", thresh)

In [ ]:
process_camera_frames(frame_threshold, 128, gval=255)

# Visualisierung

## 2D-Plots

Funktionsplots lassen sich recht einfach mit *matplotlib* erstellen:

In [ ]:
x = np.arange(-2 * math.pi, 2 * math.pi, 0.01, dtype=np.float32)
y = np.cos(x)
plt.xlim(-2 * math.pi, 2 * math.pi)
ax = plt.figure().add_subplot(111)
ax.plot(x, y)

Das Grauwert-Histogramm des Bildes von oben sieht so aus: 

In [ ]:
from PIL import Image

im = Image.open("cameraman.png")
a = np.asarray(im, dtype=np.float32)
ax = plt.figure().add_subplot(111)
h = ax.hist(a.flatten(), bins=25)

Oft möchte man die Grafiken ausführlich beschriften:

In [ ]:
import matplotlib

matplotlib.rcParams.update({"font.size": 18, "font.family": "serif"})
fig, ax = plt.subplots()
ax.plot(x, x**2, label=r"$y = \alpha^2$")
ax.plot(x, x**3, label=r"$y = \alpha^3$")
ax.set_xlabel(r"$\alpha$")
ax.set_ylabel(r"$y$")
ax.set_title("Formeln in Beschriftungen")
ax.legend(loc=4)
# lower right corner

Es ist einfach möglich, das Spektrum eines Signals auszugeben.

In [ ]:
x_min, x_max, x_step = -30.0, 30.0, 0.1
x = np.arange(x_min, x_max, x_step)
y = np.sin(x) / x
ax = plt.figure().add_subplot(111)
ax.plot(x, y)
ax.set_title("$\sin(x) / x$")

In [ ]:
M = len(x)
Y = np.abs(np.fft.fftshift(np.fft.fft(y))) ** 2.0
X = np.fft.fftshift(np.fft.fftfreq(M, x_step))
ax = plt.figure().add_subplot(111)
ax.plot(X, Y)
ax.set_title("Power-Spectrum")

Weitere interessante Beispiele für die Verwendung von pylab / matplotlib finden Sie unter [gallery](http://matplotlib.org/gallery.html) auf der [Homepage](http://matplotlib.org/). Der Source-Code für die Beispiele ist ebenfalls verfügbar, so dass sich die Gallery sehr gut als "Kopiervorlage" eignet.

##  Surface-Plots

In [ ]:
# create supporting points in polar coordinates
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.mplot3d import Axes3D

r = np.linspace(0, 1.25, 20)
p = np.linspace(0, 2 * np.pi, 20)
R, P = np.meshgrid(r, p)
# transform them to cartesian system
X, Y = R * np.cos(P), R * np.sin(P)
Z = (R**2 - 1) ** 2
fig = plt.figure()
ax = Axes3D(fig, auto_add_to_figure=False)
fig.add_axes(ax)
ax.set_zlim3d(0, 1)
ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=matplotlib.cm.YlGnBu_r)

Das Beispiel stammt aus der gallery, und zwar von [hier](http://matplotlib.org/examples/mplot3d/surface3d_radial_demo.html). Start mit

# Anpassung eines Modells an Daten

In vielen Fällen können Daten durch Polynome angenähert werden.

In [ ]:
data = np.loadtxt("polynomial.dat")
x, y = data[:, 0], data[:, 1]
ax = plt.subplots()[1]
ax.plot(x, y, "ro")

In [ ]:
x, y = data[:, 0], data[:, 1]
coeffs = np.polyfit(x, y, 3)
print("parameters: ", coeffs)
ax = plt.figure().add_subplot(111)
ax.plot(x, y, "+")
ax.plot(x, np.polyval(coeffs, x), "-")

Falls die Funktion nicht linear bezüglich der Parameter ist, muss ein iteratives Verfahren zur Anpassung verwendet werden. Meist wird die Methode von [Levenberg-Marquardt](http://ananth.in/docs/lmtut.pdf) verwendet.

In [ ]:
def func(x, a, b, c):
    return a * np.exp(-b * x) + c


x = np.linspace(0, 4, 50)
y = func(x, 2.5, 1.3, 0.5)
yn = y + 0.04 * np.random.normal(size=len(x))
ax = plt.subplots()[1]
ax.plot(x, y)
ax.plot(x, yn, "ro")

In [ ]:
from scipy.optimize import curve_fit

popt, pcov = curve_fit(func, x, yn)
print(popt)
print(pcov)

In [ ]:
y_fit = func(x, popt[0], popt[1], popt[2])
ax = plt.figure().add_subplot(111)
ax.plot(x, y_fit)
ax.plot(x, yn, "ro")

# Verarbeitung tabellarischer Daten mit pandas

Das Beispiel im Notebook *pandas.ipynb* stammt von Cyrille Rossant.

# Python und $\mu$ CT: Wurzelbehandlung

Quelle: P. Rösch, J. Jin, K.-H. Kunzelmann: *Quantitative evaluation of Root Canal Instrumentation using Skeleton Models derived from 3D Level Set Segmentation of $\mu$ CT Data.* International Journal of Computer assisted Radiology and Surgery **6 (Suppl. 1)** (2011) 209

# VTK und ParaView

Das Vizualization Toolkit ([VTK](http://www.vtk.org)) kann von Python aus benutzt werden. VTK erlaubt die Darstellung von Skalar-, Vektor- und Tensordaten und unterstützt auch die stereoskopische Ausgabe auf 3D-Monitoren und auf der "Powerwall" in M2.01. 

Beispiele finden Sie in der [VTK-Galerie](http://www.vtk.org/VTK/project/imagegallery.php).

Sie können die Visualisierungs-Pipeline auch interaktiv mit [ParaView](http://paraview.org) erstellen und
dann z.B. als Python-Code exportieren.

In [ ]:
%%file fisch.py
import vtk
#
# set up image reader
#
reader = vtk.vtkMetaImageReader()
reader.SetFileName('Carp.mhd')
reader.Update()
#
# print gray value  range
#
srange = reader.GetOutput().GetScalarRange()
print("grey value range: %.2f - %2.f" % (srange[0], srange[1]))
#
# now set up surface extractor
#
surfaceExtractor = vtk.vtkContourFilter()
surfaceExtractor.SetInputConnection(reader.GetOutputPort())
surfaceExtractor.SetValue(0, 1000)
#surfaceExtractor.SetValue(0, 1200)
surfaceExtractor.ComputeNormalsOn()
#
# simplify mesh
#
decimator = vtk.vtkQuadricDecimation()
decimator.SetInputConnection(surfaceExtractor.GetOutputPort())
decimator.SetTargetReduction(0.5)
decimator.VolumePreservationOn()
#
# extract largest connected component
#
connectivityFilter = vtk.vtkPolyDataConnectivityFilter()
connectivityFilter.SetInputConnection(decimator.GetOutputPort())
connectivityFilter.SetExtractionModeToLargestRegion()
#
# transform into triangle strips if possible
#
surfaceStripper = vtk.vtkStripper()
surfaceStripper.SetInputConnection(connectivityFilter.GetOutputPort())
surfaceMapper = vtk.vtkPolyDataMapper()
surfaceMapper.SetInputConnection(surfaceStripper.GetOutputPort())
surfaceMapper.ScalarVisibilityOff()
#
# Actor, mapper and illumination
#
surface = vtk.vtkActor()
surface.SetMapper(surfaceMapper)
surface.GetProperty().SetDiffuseColor(1, .49, .25)
surface.GetProperty().SetSpecular(.3)
surface.GetProperty().SetSpecularPower(20)
#
# finally, we need to display the stuff
#
aRenderer = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(aRenderer)
iren = vtk.vtkRenderWindowInteractor()
iren.SetInteractorStyle(vtk.vtkInteractorStyleTrackballCamera())
iren.SetRenderWindow(renWin)
aRenderer.AddActor(surface)
aRenderer.SetBackground(1, 1, 1)
#
# Interact with the data.
#
iren.Initialize()
iren.Start()

In [ ]:
%%bash
python fisch.py

# Aufgaben, freiwillig

## Numerische und symbolische Berechnungen

1. Arbeiten Sie die oben gegebenen Beispiele nochmals durch und klären Sie offene Fragen mit dem Dozenten.
1. Welche Möglichkeit gibt es, die "optimalen" Parameter einer *allgemeinen* Kurve, z.B. $$y_i = a  \cos(b x_i) \; e^{-x_i^2}$$ aus einem Satz von Meßwerten zu bestimmen? 
1. Finden Sie heraus, ob *sympy* eine der fortgeschrittenen Aufgaben aus Ihrer Mathematik-Vorlesung lösen kann und dokumentieren Sie das Beispiel in einem Jupyter-Notebook.
1. Experimentieren Sie mit [sympy gamma](http://www.sympygamma.com) und halten Sie ein für Sie interessantes Beispiel fest. Sind die angegebenen Lösungswege für Sie nachvollziehbar?

## Visualisierung


1. Verschaffen Sie sich anhander der [matplotlib gallery](http://matplotlib.org/gallery.html) einen Überblick über die Möglichkeiten, die das Paket *matplotlib* (bzw. *pylab*) bietet, kopieren Sie *zwei* für Sie interessante Beispiele in ein Notebook und erklären Sie, was in den Grafiken dargestellt wird.

# Überprüfung

1. Warum sind Python-Listen für numerische Berechnungen nicht sonderlich effizient? (max. zwei Sätze)
1. Welche Vorteile hat es, Bild-Daten als *numpy*-Arrays zu repräsentieren?
1. Erklären Sie kurz den Begriff "Symbolische Berechnungen" in Bezug auf das Paket *sympy*. (max. drei Sätze)